# Exercício Prático 03.1 - Formas 3D (Pirâmide)

- **Author:** Gabriel Van Loon
- **Data:** maio/2021
- **Objetivo:** Desenhe uma pirâmide e utilize dos comandos de mouse e teclado para realizar as transformações geométricas no objeto

    - AWSD: Translação do objeto no 2D
    - QE:   Translação no eixo Z
    - XYZ:  Rotação nos 3 eixos distintos
    - Left/Right Click do mouse: Upscalling

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

In [2]:
WINDOW_WIDTH = 600
WINDOW_HEIGHT = 600
vertices = []

Declarando os uniforms utilizados nas transformações assim como os modificores atividades/desativados pelos eventos

In [3]:
uniform_upscale   = 1.0
uniform_rotate    = [0.0, 0.0, 0.0]
uniform_translate = [0.0, 0.0, 0.0]

mod_upscale   = 0.0
mod_rotate    = [0.0, 0.0, 0.0]
mod_translate = [0.0, 0.0, 0.0]

### Funções para definição de Janelas

Encapsula as funções que criam a única e principal janela do programa.

In [4]:
def configure_window():
    glfw.init()
    glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
    glfw.window_hint(glfw.RESIZABLE, glfw.FALSE)
    window = glfw.create_window(WINDOW_WIDTH, WINDOW_HEIGHT, "Computer Graphics 101", None, None)
    glfw.make_context_current(window)
    return window

# Starting window
window = configure_window()

### Classe para Criação de Shaders

Encapsula as funções que permitem criar e utilizar um determinado shader

In [5]:
class Shader:
    def __init__(self, vertex_code = "", fragment_code = ""):
        self.program = glCreateProgram()
        
        # Create the vertex and shader program
        vertex   = glCreateShader(GL_VERTEX_SHADER)
        fragment = glCreateShader(GL_FRAGMENT_SHADER)
        
        # Set shaders sources code
        glShaderSource(vertex, vertex_code)
        glShaderSource(fragment, fragment_code)
        
        # Compiling vertex shader
        glCompileShader(vertex)
        if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(vertex).decode()
            print(error)
            raise RuntimeError("Erro de compilacao do Vertex Shader")
        
        # Compile fragment shader
        glCompileShader(fragment)
        if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(fragment).decode()
            print(error)
            raise RuntimeError("Erro de compilacao do Fragment Shader")
            
        # If success atach the compiled codes to the program
        glAttachShader(self.program, vertex)
        glAttachShader(self.program, fragment)
        
        # Build program
        glLinkProgram(self.program)
        if not glGetProgramiv(self.program, GL_LINK_STATUS):
            print(glGetProgramInfoLog(self.program))
            raise RuntimeError('Linking error')
            
        # Delete shaders (we don't need them anymore after compile)
        glDeleteShader(vertex);
        glDeleteShader(fragment)
    
    def use(self):
        glUseProgram(self.program)
        
        # Define the attributes mapping from buffer (stride = vertices.strides[0] = 12)
        attr_loc = glGetAttribLocation(self.program, "position")
        glEnableVertexAttribArray(attr_loc)
        glVertexAttribPointer(attr_loc, 3, GL_FLOAT, False, 12, ctypes.c_void_p(0))
        
        # Define the uniforms in the shader
        loc = glGetUniformLocation(self.program, "u_upscale");
        glUniform1f(loc, uniform_upscale);
        
        loc = glGetUniformLocation(self.program, "u_rotate");
        glUniform3f(loc, uniform_rotate[0], uniform_rotate[1], uniform_rotate[2]);
        
        loc = glGetUniformLocation(self.program, "u_translate");
        glUniform3f(loc, uniform_translate[0], uniform_translate[1], uniform_translate[2])
        
    def setFloat(self, name, value):
        loc = glGetUniformLocation(self.program, name)
        glUniform1f(loc, value)
        
    def set3Float(self, name, value):
        loc = glGetUniformLocation(self.program, name)
        glUniform3f(loc, value[0], value[1], value[2])

### Shader 1 - Pirâmide

In [6]:
vertex_code = """
    attribute vec3 position;
    varying   vec3 fPosition;
    
    uniform float u_upscale;
    uniform vec3  u_rotate;
    uniform vec3  u_translate;
    
    void main(){ 
       // Upscaling 
       mat4 upscale_matrix = mat4(
           u_upscale, 0.0, 0.0, 0.0,   // first column
           0.0, u_upscale, 0.0, 0.0,   // second column        
           0.0, 0.0, u_upscale, 0.0,   
           0.0, 0.0, 0.0,       1.0
       );
       
       // Rotate in Z
       mat4 z_rotate_matrix = mat4(
           cos(u_rotate.z),  sin(u_rotate.z), 0.0, 0.0,   // first column
           -sin(u_rotate.z), cos(u_rotate.z), 0.0, 0.0,   // second column
           0.0, 0.0, 1.0, 0.0,                        // last column
           0.0, 0.0, 0.0, 1.0
       );
       
       // Rotate in X
       mat4 x_rotate_matrix = mat4(
           1.0, 0.0, 0.0, 0.0, 
           0.0, cos(u_rotate.x),  sin(u_rotate.x), 0.0,  
           0.0, -sin(u_rotate.x), cos(u_rotate.x), 0.0,                    
           0.0, 0.0, 0.0, 1.0
       );
       
       // Rotate in Y
       mat4 y_rotate_matrix = mat4(
           cos(u_rotate.y),  0.0,  sin(u_rotate.y), 0.0,   
           0.0, 1.0, 0.0, 0.0,
           -sin(u_rotate.y), 0.0, cos(u_rotate.y), 0.0,
           0.0, 0.0, 0.0, 1.0
       );
       
       // Translate
       mat4 translate_matrix = mat4(
           1.0, 0.0, 0.0, 0.0,   
           0.0, 1.0, 0.0, 0.0,
           0.0, 0.0, 1.0, 0.0,
           u_translate.x, u_translate.y, u_translate.z, 1.0
       );

       // Aplying transformations (Upscale + Rotate + Translate)
       vec4 coord =  vec4(position, 1.0);
       coord  =  upscale_matrix * coord;
       coord  =  x_rotate_matrix * coord;
       coord  =  y_rotate_matrix * coord;
       coord  =  z_rotate_matrix * coord;
       coord  =  translate_matrix * coord;
       
       gl_Position =  coord;
       fPosition   = gl_Position;
    }
"""

frag_code = """
    varying vec3 fPosition;
    uniform vec3 u_color;
    
    void main(){
        // gl_FragColor = vec4(fPosition, 1.0);
        gl_FragColor = vec4(u_color, 1.0);
    }
"""

object_shader = Shader(vertex_code, frag_code)

In [7]:
object_offset = len(vertices)
object_qtd_vertices = 18
vertices += [
    # Bottom Square 1/2
    (-0.5,  0.0,  0.5),   
    (-0.5,  0.0, -0.5),  
    (+0.5,  0.0, +0.5),
    
    # Bottom Square 2/2
    (-0.5,  0.0, -0.5),  
    (+0.5,  0.0, +0.5),
    (+0.5,  0.0, -0.5),
    
    # Front Face
    (-0.5,  0.0, +0.5),  
    (+0.5,  0.0, +0.5),
    ( 0.0,  1.0,  0.0),
    
    # Left Face
    (-0.5,  0.0, +0.5),  
    (-0.5,  0.0, -0.5),
    ( 0.0,  1.0,  0.0),
    
    # Right Face
    (+0.5,  0.0, +0.5),  
    (+0.5,  0.0, -0.5),
    ( 0.0,  1.0,  0.0),
    
    # Back Face
    (-0.5,  0.0, -0.5),  
    (+0.5,  0.0, -0.5),
    ( 0.0,  1.0,  0.0),
]

In [8]:
# Generate face colors
face_color = []
for i in range(6):
    face_color.append([np.random.random(), np.random.random(), np.random.random()])

def draw_object(vertices):
    global object_shader
    
    # Select the shader
    object_shader.use()
    
    for i in range(2, 6):
        object_shader.set3Float('u_color', face_color[i])
        glDrawArrays(GL_TRIANGLES, object_offset + (3*i), 3)
    
    object_shader.set3Float('u_color', face_color[0])
    glDrawArrays(GL_TRIANGLES, 0, 6)

### Capturando Eventos do Teclado e Mouse

- **Mouse button references:** https://www.glfw.org/docs/latest/group__buttons.html
- **Keyboard button references:** https://www.glfw.org/docs/latest/group__keys.html
- **Modifiers: https://www.glfw.org/docs/latest/group__mods.html

In [9]:
# mod_upscale   = 0.0
# mod_rotate    = [0.0, 0.0, 0.0]
# mod_translate = [0.0, 0.0, 0.0]

# Action is 1 when press-in and 0 in press-out

def key_event(window, key, scancode, action, mods):
    
    global mod_upscale, mod_rotate, mod_translate

    # Translate 2D (X,Y)
    translate_diff = 0.00005
    if key == glfw.KEY_W:
        mod_translate[1] = translate_diff * action
    if key == glfw.KEY_S:
        mod_translate[1] = -translate_diff * action
    if key == glfw.KEY_A:
        mod_translate[0] = -translate_diff * action
    if key == glfw.KEY_D:
        mod_translate[0] = translate_diff * action
    
    # Translate Z
    if key == glfw.KEY_Q:
        mod_translate[2] = translate_diff * action
    if key == glfw.KEY_E:
        mod_translate[2] = -translate_diff * action
    
    # Rotate X,Y or Z clockwise
    rotate_diff = 0.0005
    if key == glfw.KEY_X:
        mod_rotate[0] = rotate_diff * action
    if key == glfw.KEY_Y:
        mod_rotate[1] = rotate_diff * action
    if key == glfw.KEY_Z:
        mod_rotate[2] = rotate_diff * action
    
        
def mouse_event(window, button, action, mods):
    
    global mod_upscale, mod_rotate, mod_translate
    
    # Uniform Upscale
    upscale_diff = 0.00005
    if button == glfw.MOUSE_BUTTON_LEFT:
        mod_upscale = upscale_diff * action 
    elif button == glfw.MOUSE_BUTTON_RIGHT:
        mod_upscale = -upscale_diff * action
        
    
glfw.set_key_callback(window, key_event)
glfw.set_mouse_button_callback(window, mouse_event)

In [10]:
# Function that update uniforms with the modifiers
def update_uniforms():
    global mod_upscale, mod_rotate, mod_translate
    global uniform_upscale, uniform_rotate, uniform_translate
    
    uniform_upscale   += mod_upscale
    uniform_rotate[0] += mod_rotate[0]
    uniform_rotate[1] += mod_rotate[1]
    uniform_rotate[2] += mod_rotate[2]
    uniform_translate[0] += mod_translate[0]
    uniform_translate[1] += mod_translate[1]
    uniform_translate[2] += mod_translate[2]

### Iniciando o Programa e suas variáveis

A primeira coisa que precisamos fazer é iniciar o buffer e enviar os vértices para a GPU

In [11]:
# Convertendo para numpy array
vertices = np.array(vertices, dtype=np.float32)

In [12]:
# Gerando um Buffer para o Array de vértices
buffer = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Enviando os dados definindo o buffer como estatico, pois não pretendemos a
# posição dos vértices iniciais (GL_DYNAMIC_DRAW | GL_STATIC_DRAW)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)

In [13]:
# Show the window
glfw.show_window(window)

# Active 3D 
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(window):
    glfw.poll_events() 
    
    # Reset the screen with the white color
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # Apply events modifiers
    update_uniforms()
    
    # Polygon form
    # glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
    
    # Draw objects
    draw_object(vertices)
    
    glfw.swap_buffers(window)
glfw.terminate()

### Referencias de matrizes

A seguir apenas um referencial de como montar as matrizes de transformação

#### a) Matriz de Translação

$$
\begin{pmatrix}
x_f \\ y_f \\ z_f \\ h_f \\
\end{pmatrix} 
= 
\begin{pmatrix}
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & 1 & t_z\\
0 & 0 & 0 &1 \\
\end{pmatrix} 
*
\begin{pmatrix}
x_0 \\ y_0 \\ z_0 \\ h_0 \\
\end{pmatrix} 
$$

#### b) Matriz de Escala 2D (z=0)

$$
\begin{pmatrix}
x_f \\ y_f \\ z_f \\ 1
\end{pmatrix} 
= 
\begin{pmatrix}
s_x & 0 & 0 & 0  \\
0   & s_y & 0 & 0\\
0   & 0 & s_z & 0\\
0   & 0 & 0 & 1 \\
\end{pmatrix} 
*
\begin{pmatrix}
x_0 \\ y_0 \\ z_0 \\ 1
\end{pmatrix} 
$$

#### c) Matriz de Rotação em Z (Padrão 2D)

$$
\begin{pmatrix}
x_f \\ y_f \\ z_f \\ 1
\end{pmatrix} 
= 
\begin{pmatrix}
\cos(\theta_z) & -\sin(\theta_z) & 0 & 0\\
\sin(\theta_z) & \cos(\theta_z) & 0  & 0\\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{pmatrix} 
*
\begin{pmatrix}
x_0 \\ y_0 \\ z_0 \\ 1
\end{pmatrix} 
$$

#### d) Matriz de Rotação em X

$$
\begin{pmatrix}
x_f \\ y_f \\ z_f \\ 1
\end{pmatrix} 
= 
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & \cos(\theta_x) & -\sin(\theta_x) &  0\\
0 & \sin(\theta_x) & \cos(\theta_x) &  0\\
0 & 0 & 0 & 1 \\
\end{pmatrix} 
*
\begin{pmatrix}
x_0 \\ y_0 \\ z_0 \\ 1
\end{pmatrix} 
$$

#### d) Matriz de Rotação em Y

$$
\begin{pmatrix}
x_f \\ y_f \\ z_f \\ 1
\end{pmatrix} 
= 
\begin{pmatrix}
\cos(\theta_y) & 0 & -\sin(\theta_y) & 0 \\
0 & 1 & 0 &  0\\
\sin(\theta_y) & 0 & \cos(\theta_y) &  0\\
0 & 0 & 0 & 1 \\
\end{pmatrix} 
*
\begin{pmatrix}
x_0 \\ y_0 \\ z_0 \\ 1
\end{pmatrix} 
$$